# Embedded ML - Lab 1.1: Native implementation of Artificial Neural Netwroks

In this lab you are asked to write the code for an Artificial Neural Network (ANN) without using ML libraries such as SciKit-learn, PyTorch or TensorFlow, but you are allowed to use standard libraries such as math, numpy and matplotlib if needed. You are given some code but you are expected to write some more and be able to explain and modify everything. This is a key foundational exercise for you to understand the efficiency aspects that will be dealt with throughout this course.

### Learning outcomes


* Explain the basic concepts of ANNs
* Implement simple ANNs in Python without using advanced libraries
* Analyze the computational resources demanded when training, running inference and scaling ANNs



### 1. Linear regression
Linear regression is perhaps the simplest form of ML and can be thought of as an ANN with a single neuron. Yet, it can make a linear approximation of an input-output pair of data arrays.

Below is an incomplete code for a Python class that implements a linear regressor. You should **complete the missing code** for the predict() and error() methods and then write a simple implementation of the class.

In [ ]:
# Solution to Linear Regression excersize

# Ref 1: https://github.com/tinyMLx/colabs/blob/master/2-1-4-ExploringLoss.ipynb
# Ref 2: https://github.com/tinyMLx/colabs/blob/master/2-1-6-MimimizingLoss.ipynb

import math
import random

class LinRegressor:
  def __init__(self, w, b):
    self.w = w
    self.b = b

  def predict(self, x):
    self.myY = []

    for thisX in x:
      thisY = self.w * thisX + self.b
      self.myY.append(thisY)

    return self.myY

  def error(self, y):
    total_square_error = 0
    for i in range(0, len(y)):
      square_error = (y[i] - self.myY[i]) ** 2
      total_square_error += square_error

    return math.sqrt(total_square_error)

dataset_A = {
    # y = 2x - 1
    "x": [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8],
    "y": [-3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    }

dataset_B = {
    # y = ((x/5) + 1)^2 - 3
    "x": [-8, -5, -3.4, -2, 0, 1.9, 4, 6.2, 8, 11.5],
    "y": [-2.64, -3, -2.9, -2.64, -2, -1.09, 0.24, 2.01, 3.76, 7.89]
    }

w = 2 #random.randint(-10, 10)
b = -1 #random.randint(-10, 10)

x = dataset_A["x"]
y = dataset_A["y"]

model = LinRegressor(w, b)

print("Real Y is " + str(y))
print("My Y is   " + str(model.predict(x)))
print("My loss is: " + str(model.error(y)))

Real Y is [-3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
My Y is   [-3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
My loss is: 0.0


Measure the error for three different sets of parameter values, for each dataset. **Plot the datasets against the predictions** and analyze the model results obtained.

*   Can the error of dataset A be zero?
*   Can the error of dataset B be zero?
*   A zero error means that the model represents the system perfectly?
*   Can you model any kind of system with this type of model?

### 2. Artificial Neural Networks
Based on the principles of aproximating the mathematical relationship between two arrays of data, ANNs are scaled up algorithms that connect multiple linear regressors with activation functions in order to detect more complex relationships between data. The computation elements that make up an ANN are called Perceptrons or simply neurons, and they are topologically organized in layers.

Given is a Python code that partially implements a neural network with three layers: input, hidden and output. It defines methods for training and inference and uses the XOR function as a test case.

Study the code to get familiar with it and **complete the implementation of the forward()** method that takes in the network inputs to produce the outputs.
Verify the network works by running the code and observing the error going down and producing corrects results. Also play with the training parameters to see how learning improves or degrades.

Then mode to **implementing the my_dot() method to replace NumPy's dot()**, in order to make explicit the operations that are executed every time the method is called. Modify the forward method to use the new function and verify its correctness.

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_input_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_hidden_output = np.zeros((1, self.output_size))

    def my_dot(self, A, B):
        """
        Custom implementation of the dot product between two matrices or vectors.
        """
        A = np.atleast_2d(A)  # Ensure A is at least 2D
        B = np.atleast_2d(B)  # Ensure B is at least 2D

        rows_A, cols_A = A.shape
        rows_B, cols_B = B.shape

        # Ensure matrices can be multiplied
        if cols_A != rows_B:
            raise ValueError("Incompatible matrix dimensions for multiplication")

        # Initialize result matrix with zeros
        result = np.zeros((rows_A, cols_B))

        # Perform matrix multiplication
        for i in range(rows_A):
            for j in range(cols_B):
                for k in range(cols_A):
                    result[i][j] += A[i, k] * B[k, j]

        return result if result.shape[0] > 1 else result.flatten()  # Return a flattened array if it's a single row


    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        # Forward propagation through the network
        self.hidden_output = self.sigmoid(self.my_dot(x, self.weights_input_hidden) + self.bias_input_hidden)
        self.output = self.sigmoid(self.my_dot(self.hidden_output, self.weights_hidden_output) + self.bias_hidden_output)
        return self.output

    def backward(self, x, y, output, learning_rate):
        # Backpropagation and weight updates
        self.error = y - output
        d_output = self.error * self.sigmoid_derivative(output)

        self.hidden_error = self.my_dot(d_output, self.weights_hidden_output.T)
        d_hidden = self.hidden_error * self.sigmoid_derivative(self.hidden_output)

        self.weights_hidden_output += self.my_dot(self.hidden_output.T, d_output) * learning_rate
        self.bias_hidden_output += np.sum(d_output, axis=0, keepdims=True) * learning_rate
        self.weights_input_hidden += self.my_dot(x.T, d_hidden) * learning_rate
        self.bias_input_hidden += np.sum(d_hidden, axis=0, keepdims=True) * learning_rate

    def train(self, x, y, epochs, learning_rate):
        error = 0
        for epoch in range(epochs):
            output = self.forward(x)
            self.backward(x, y, output, learning_rate)
            if epoch % 100 == 0:
                error = np.mean(np.square(y - output))
                print(f'Epoch {epoch}: Loss = {error:.4f}')

# Define XOR dataset
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])

# Initialize and train the neural network
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
nn.train(X, y, epochs=1000, learning_rate=0.5)

# Test the trained model
print("\nTest the trained model:")
for i in range(len(X)):
    output = nn.forward(X[i])
    print(f"Input: {X[i]}, Predicted Output: {output}, Actual Output: {y[i]}")


Epoch 0: Loss = 0.2745
Epoch 100: Loss = 0.2452
Epoch 200: Loss = 0.2323
Epoch 300: Loss = 0.2060
Epoch 400: Loss = 0.1717
Epoch 500: Loss = 0.1299
Epoch 600: Loss = 0.0822
Epoch 700: Loss = 0.0473
Epoch 800: Loss = 0.0282
Epoch 900: Loss = 0.0182

Test the trained model:
Input: [0 0], Predicted Output: [[0.10479382]], Actual Output: [0]
Input: [0 1], Predicted Output: [[0.87874124]], Actual Output: [1]
Input: [1 0], Predicted Output: [[0.90248499]], Actual Output: [1]
Input: [1 1], Predicted Output: [[0.12589268]], Actual Output: [0]


Let's define an abstraction in which basic computations are: additions, subtractions, multiplications, divisions or computing an activation fuction such as the sigmoid or its derivative. Then, analyze the code in detail to answer the following questions:

*   How many scalar basic computations are requiered for one forward pass, for one training iteration and for a complete training process?
*   Which are the newtwork parameters that determine the amount of computations required?

**Write a formula** that gives the amount of basic scalar computations depending on the network parameters.

### 3. Scaling ANNs

In manys cases, but not all, increasing the number of layers and the number of neurons per layer leads to a higher accuracy of the model. This comes at the expense of more resources needed to run the network: memory and computation. And ultimately, it can lead to a higher application latency and energy consumption.

Here you should create a fully-connected neural network based on the previous model, this time to classify handwritten numbers using the **MNIST dataset**. Investigate how to obtain the dataset and how to prepare a proper partition between training and test.

The number of input neurons must be equal to the number of pixels on each image (depending on the chosen resolution). The number of output neurons must be 10, since there are 10 diffirent digits we want to classify. A new method must be included to select which of the digits was identified (by finding the most active output neuron). **Configure and test at least five versions of the model** by varying the amount of neurons in the hidden layer.

Make a table or a plot to report the following for each model:

*   Number of model parameters
*   Number of basic scalar computations for a forward pass (using the previously created formula)
*   Execution time for training and for a forward pass
*   Model's Top-1 accuracy.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/Classroom/Embedded Machine Learning 2025-1/Labs/Solved labs/
%ls

import numpy as np
import gzip

def load_mnist_images(filename):

    with open(filename, 'rb') as f:
        data = np.fromfile(f, dtype=np.uint8, offset=16)
    data = data.reshape(-1, 28 * 28) / 255.0
    return data

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        data = np.fromfile(f, dtype=np.uint8, offset=8)
    return data

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_input_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden_output = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        # Forward propagation through the network
        self.hidden_output = self.sigmoid(np.dot(x, self.weights_input_hidden) + self.bias_input_hidden)
        self.output = self.sigmoid(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_hidden_output)
        return self.output

    def backward(self, x, y):
        # Backpropagation and weight updates
        error = y - self.output
        d_output = error * self.sigmoid_derivative(self.output)

        hidden_error = d_output.dot(self.weights_hidden_output.T)
        d_hidden = hidden_error * self.sigmoid_derivative(self.hidden_output)

        self.weights_hidden_output += self.hidden_output.T.dot(d_output) * self.learning_rate
        self.bias_hidden_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
        self.weights_input_hidden += x.T.dot(d_hidden) * self.learning_rate
        self.bias_input_hidden += np.sum(d_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            for i in range(len(X)):
                self.forward(X[i])
                self.backward(X[i], y[i])
            if epoch % 10 == 0:
                loss = np.mean(np.square(y - self.output))
                print(f'Epoch {epoch}: Loss = {loss:.4f}')

# Load MNIST data
X_train = load_mnist_images('train-images.idx3-ubyte')
y_train = load_mnist_labels('train-labels.idx1-ubyte')
X_test = load_mnist_images('t10k-images.idx3-ubyte')
y_test = load_mnist_labels('t10k-labels.idx1-ubyte')

# Preprocess data
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

# Initialize and train the neural network
input_size = 784  # 28x28 pixels
hidden_size = 100
output_size = 10  # 10 classes (digits 0-9)
nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate=0.1)
nn.train(X_train, y_train, epochs=100)

# Test the trained model
correct = 0
for i in range(len(X_test)):
    output = nn.forward(X_test[i:i+1])
    prediction = np.argmax(output)
    if prediction == np.argmax(y_test[i]):
        correct += 1

accuracy = correct / len(X_test)
print(f'Test Accuracy: {accuracy:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Classroom/Embedded Machine Learning 2025-1/Labs/Solved labs
 3-5-13-PretrainedModel.ipynb
 4-2-12-OV7675ImageViewer.ipynb
'Embedded ML - Lab 1.1 Native implementation of Artificial Neural Networks SOLUTION.ipynb'
'Embedded ML - Lab 1.1_ Neural Networks in Python SOLUTION.ipynb'
'Embedded ML - Lab 1.2_ Model Compression SOLUTION.ipynb'
'Embedded ML - Lab 2.1_ TensorFlow SOLUTION.ipynb'
'Embedded ML - Lab 2.2_ TensorFlow Lite SOLUTION.ipynb'
'Embedded ML - Lab 2.3_ TensorFlow Lite Micro SOLUTION.ipynb'
'Embedded ML - Lab 3_ ML on Embedded GPUs SOLUTION'
'Embedded ML - Lab 4_ ML App Design.docx'
 t10k-images.idx3-ubyte
 t10k-labels.idx1-ubyte
 train-labels.idx1-ubyte


FileNotFoundError: [Errno 2] No such file or directory: 'train-images.idx3-ubyte'